# Fuse metadata for Chicago paper

We've got reviewed books, prizewinners, and bestsellers. We need to merge the three frames and assign unique author IDs.

We also need to get some unreviewed books as a control set.

In [77]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [2]:
review = pd.read_csv("cohortmatches.tsv", sep = '\t')
review.head()

,volume_author,review_author,aRatio,volume_title,trunc_revtitle,review_title,tRatio,volume_year,review_year,docid,rownum
0,"shaw, irwin","shaw, irwin.",96,"welcome to the city, and other","welcome to the city, and other","welcome to the city, and other stories.",106,1942,1942,uc1.b3347126,3289
1,"bennett, arnold","bennett, arnold.",97,woman who stole everything,"woman who stole everything, an","woman who stole everything, and other stories.",106,1927,1927,uc1.$b322688,4140
2,"lippincott, joseph wharton","lippincott, joseph wharton.",98,wilderness champion; the story,wilderness champion; the story,wilderness champion; the story of a great houn...,106,1944,1944,uc1.b3407035,3481
3,"becker, may lamberton","becker, mrs may (lamberton), ed.",79,golden tales of the southwest;,golden tales of the southwest;,golden tales of the southwest; selected with a...,106,1939,1939,uc1.$b116192,4283
4,"miller, alice duer, mrs","miller, mrs alice (duer).",75,instruments,"instruments of darkness, and o","instruments of darkness, and other stories.",106,1926,1926,mdp.39015059415383,2405


In [4]:
best = pd.read_csv("bestsellersincohortmodel.tsv", sep = '\t')
best.head()

,titlemeta_ID,titlemeta_author,titlemeta_title,bestseller_author,bestseller_title,bestseller_year,ratio_author,ratio_title,docid,year
0,uc2.ark+=13960=t05x25z6x,"lewis, sinclair",Main street,"lewis, sinclair",Main Street,1921,100,100,uc2.ark+=13960=t05x25z6x,1920
1,mdp.39015000636111,"lewis, sinclair",Arrowsmith,"lewis, sinclair",Arrowsmith,1925,100,100,mdp.39015000636111,1925
2,mdp.39015008361134,"lewis, sinclair",Elmer Gantry,"lewis, sinclair",Elmer Gantry,1927,100,100,mdp.39015008361134,1927
3,mdp.39015048880846,"lewis, sinclair",Dodsworth,"lewis, sinclair",Dodsworth,1929,100,100,mdp.39015048880846,1929
4,mdp.39015000636095,"lewis, sinclair",Ann Vickers,"lewis, sinclair",Ann Vickers,1933,100,100,mdp.39015000636095,1933


In [5]:
prize = pd.read_csv("prizewinnersincohortmodel.tsv", sep = '\t')
prize.head()

,titlemeta_ID,titlemeta_author,titlemeta_title,prize_author,prize_title,prize_date,ratio_author,ratio_title,docid,year
0,mdp.39015005332708,"Barnes, Margaret Ayer",Years of grace,"Barnes, Margaret Ayer",Years of Grace,1931,100,100,mdp.39015005332708,1930
1,mdp.39015005305936,"Barnes, Margaret Ayer",Westward passage,"Barnes, Margaret Ayer",Years of Grace,1931,100,40,mdp.39015005305936,1931
2,uc1.$b56317,"Barnes, Margaret Ayer",Within this present,"Barnes, Margaret Ayer",Years of Grace,1931,100,24,uc1.$b56317,1933
3,mdp.39015063551983,"Barnes, Margaret Ayer","Edna, his wife, an American idyll","Barnes, Margaret Ayer",Years of Grace,1931,100,34,mdp.39015063551983,1935
4,mdp.39015051324153,"Barnes, Margaret Ayer",Wisdom's gate,"Barnes, Margaret Ayer",Years of Grace,1931,100,44,mdp.39015051324153,1938


### goal

what we want is a dataframe with

    tm_author, tm_title, rev_title (if exists), docid, rownum (if exists), 
    review_year (if exists), bestseller_year (if exists), tm_year,
    lowest_year, prizewinning_author (T/F), bestseller (T/F)
    

In [6]:
review = review.rename(columns = {'volume_author': 'tm_author', 'volume_title': 'tm_title', 'volume_year': 'tm_year'})
review = review.loc[: , ['docid', 'tm_author', 'tm_title', 'tm_year', 'review_year', 'rownum', 'review_title']]
review.head()

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942,1942,3289,"welcome to the city, and other stories."
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927,1927,4140,"woman who stole everything, and other stories."
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944,1944,3481,wilderness champion; the story of a great houn...
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939,1939,4283,golden tales of the southwest; selected with a...
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926,1926,2405,"instruments of darkness, and other stories."


In [7]:
best = best.rename(columns = {'year': 'tm_year', 'titlemeta_author': 'tm_author', 'titlemeta_title': 'tm_title'})
best = best.loc[: , ['docid', 'tm_author', 'tm_title', 'tm_year', 'bestseller_year']]
best.tm_title = best.tm_title.str.lower()
best.head()

,docid,tm_author,tm_title,tm_year,bestseller_year
0,uc2.ark+=13960=t05x25z6x,"lewis, sinclair",main street,1920,1921
1,mdp.39015000636111,"lewis, sinclair",arrowsmith,1925,1925
2,mdp.39015008361134,"lewis, sinclair",elmer gantry,1927,1927
3,mdp.39015048880846,"lewis, sinclair",dodsworth,1929,1929
4,mdp.39015000636095,"lewis, sinclair",ann vickers,1933,1933


In [8]:
best['is_bestseller'] = True

In [9]:
prize = prize.rename(columns = {'year': 'tm_year', 'titlemeta_author': 'tm_author', 'titlemeta_title': 'tm_title'})
prize = prize.loc[: , ['docid', 'tm_author', 'tm_title', 'tm_year']]
prize['prize_author'] = True
prize.head()

,docid,tm_author,tm_title,tm_year,prize_author
0,mdp.39015005332708,"Barnes, Margaret Ayer",Years of grace,1930,True
1,mdp.39015005305936,"Barnes, Margaret Ayer",Westward passage,1931,True
2,uc1.$b56317,"Barnes, Margaret Ayer",Within this present,1933,True
3,mdp.39015063551983,"Barnes, Margaret Ayer","Edna, his wife, an American idyll",1935,True
4,mdp.39015051324153,"Barnes, Margaret Ayer",Wisdom's gate,1938,True


In [10]:
prize.tm_author = prize.tm_author.str.lower()
prize.tm_title = prize.tm_title.str.lower()
prize.head()

,docid,tm_author,tm_title,tm_year,prize_author
0,mdp.39015005332708,"barnes, margaret ayer",years of grace,1930,True
1,mdp.39015005305936,"barnes, margaret ayer",westward passage,1931,True
2,uc1.$b56317,"barnes, margaret ayer",within this present,1933,True
3,mdp.39015063551983,"barnes, margaret ayer","edna, his wife, an american idyll",1935,True
4,mdp.39015051324153,"barnes, margaret ayer",wisdom's gate,1938,True


In [25]:
complete = review.merge(prize, how = 'outer', left_on = ['docid', 'tm_author'], right_on = ['docid', 'tm_author'])
print(review.shape, prize.shape)
complete.shape

(4412, 7) (365, 5)


(4549, 10)

In [26]:
complete.head()

,docid,tm_author,tm_title_x,tm_year_x,review_year,rownum,review_title,tm_title_y,tm_year_y,prize_author
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",NaN,NaN,NaN
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",NaN,NaN,NaN
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,NaN,NaN,NaN
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,NaN,NaN,NaN
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",NaN,NaN,NaN


In [27]:
listofdicts = []

for idx, row in complete.iterrows():
    newdict = dict()
    for col in complete.columns:
        if col.endswith('_x') or col.endswith('_y'):
            prefix = col[0: -2]
            if pd.isnull(row[prefix + '_x']):
                value = row[prefix + '_y']
            elif pd.isnull(row[prefix + '_y']):
                value = row[prefix + '_x']
            else:
                if type(row[prefix + '_x']) == float or type(row[prefix + '_y']) == float:
                    value = row[prefix + '_x']
                elif len(row[prefix + '_x']) > len(row[prefix + '_y']):
                    value = row[prefix + '_x']
                else:
                    value = row[prefix + '_y']
                    
            newdict[prefix] = value
        else:
            newdict[col] = row[col]
        
    listofdicts.append(newdict)
complete = pd.DataFrame(listofdicts)
print(complete.shape)
complete.head()

(4549, 8)


,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",NaN
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",NaN
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,NaN
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,NaN
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",NaN


In [28]:
print(complete.shape)
complete = complete.merge(best, how = 'outer', left_on = ['docid', 'tm_author'], right_on = ['docid', 'tm_author'])
print(best.shape)
complete.shape

(4549, 8)
(187, 6)


(4620, 12)

In [29]:
complete.head()

,docid,tm_author,tm_title_x,tm_year_x,review_year,rownum,review_title,prize_author,tm_title_y,tm_year_y,bestseller_year,is_bestseller
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",NaN,NaN,NaN,NaN,NaN
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",NaN,NaN,NaN,NaN,NaN
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,NaN,NaN,NaN,NaN,NaN
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,NaN,NaN,NaN,NaN,NaN
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",NaN,NaN,NaN,NaN,NaN


In [30]:
listofdicts = []

for idx, row in complete.iterrows():
    newdict = dict()
    for col in complete.columns:
        if col.endswith('_x') or col.endswith('_y'):
            prefix = col[0: -2]
            if pd.isnull(row[prefix + '_x']):
                value = row[prefix + '_y']
            elif pd.isnull(row[prefix + '_y']):
                value = row[prefix + '_x']
            else:
                if type(row[prefix + '_x']) == float or type(row[prefix + '_y']) == float:
                    value = row[prefix + '_x']
                elif len(row[prefix + '_x']) > len(row[prefix + '_y']):
                    value = row[prefix + '_x']
                else:
                    value = row[prefix + '_y']
                    
            newdict[prefix] = value
        else:
            newdict[col] = row[col]
        
    listofdicts.append(newdict)
complete = pd.DataFrame(listofdicts)
print(complete.shape)
complete.head()

(4620, 10)


,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",NaN,NaN,NaN
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",NaN,NaN,NaN
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,NaN,NaN,NaN
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,NaN,NaN,NaN
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",NaN,NaN,NaN


In [31]:
len(set(complete.docid))

4554

In [34]:
complete = complete.drop_duplicates()
complete.shape

(4576, 10)

In [36]:
sum(complete.prize_author == True)

327

In [37]:
sum(complete.is_bestseller == True)

187

In [40]:
complete.is_bestseller = complete.is_bestseller.fillna(value = False)

In [42]:
complete.prize_author = complete.prize_author.fillna(value = False)

In [43]:
complete.iloc[4390:4440, :]

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller
4426,nyp.33433076049414,"perry, lawrence",for the ga,1920.0,1920.0,2480.0,for the e's sake.,False,NaN,False
4427,uc1.$b80847,"duhamel, georges",salavin,1935.0,1936.0,826.0,salavln; tr.,False,NaN,False
4428,nyp.33433074807599,"brent, john",a man's game,1882.0,1921.0,164.0,man's game.,False,NaN,False
4429,inu.30000035052244,"maugham, w. somerset, (william somerset)",casuarina tree : six stories,1926.0,1926.0,1237.0,ciisuarina tree.,False,NaN,False
4430,mdp.39015030946902,"bodenheim, maxwell",georgie may,1928.0,1928.0,10.0,qeorgle may.,False,NaN,False
4431,mdp.49015000570912,"clark, walter van tilburg","watchful gods, and other st",1950.0,1950.0,1024.0,watchful and other stories.,False,NaN,False
4432,uc1.b3688341,"frye, ralph bridges",uncle 'lish,1945.0,1945.0,268.0,uncle 'ush.,False,NaN,False
4433,uc1.b3148807,"kaus, gina",tomorrow we,1933.0,1934.0,4996.0,tomorrow we part [tr.,False,NaN,False
4434,mdp.39015015186334,"green, henry","loving,",1945.0,1949.0,3901.0,loving.,False,NaN,False
4435,uc1.b4104150,"bacon, leonard",furioso,1932.0,1932.0,2061.0,the furloso.,False,NaN,False


In [44]:
meta = pd.read_csv("../../cohort/topicdata/comprehensivebooktopicdata.tsv", sep="\t", low_memory=False)

In [46]:
meta = meta.loc[ : , ['docid', 'birthyear', 'firstpub', 'hathi_author', 'hathi_title', 'us_national', 'authof3ormore', 'age']]
meta = meta.rename(columns = {'hathi_author': 'tm_author', 'hathi_title': 'tm_title'})
meta.head()

,docid,birthyear,firstpub,tm_author,tm_title,us_national,authof3ormore,age
0,uc1.b4975632,NaN,1989,"Aakhus, Patricia",The voyage of Mael Duin's curragh,True,False,NaN
1,inu.30000112046630,1911.0,1957,"Abbe, George",The winter house,False,True,46.0
2,uc1.$b799882,1911.0,1967,"Abbe, George",The funeral,False,True,56.0
3,uc1.$b149331,1911.0,1968,"Abbe, George",Yonderville,False,True,57.0
4,uc1.32106007981415,1927.0,1956,"Abbey, Edward",The brave cowboy : an old tale in a new time,True,True,29.0


In [ ]:
meta.tm_author = meta.tm_author.str.lower()
meta.tm_title = meta.tm_title.str.lower()

In [47]:
wehave = set(complete.docid)
toconcat = []

for year in range(1916, 1951):
    df = meta.loc[meta.firstpub == year, :]
    df = df.loc[~df.docid.isin(wehave), : ]
    toconcat.append(df.sample(30))
    
missing = pd.concat(toconcat)
missing.shape

(1050, 8)

In [48]:
missing.head()

,docid,birthyear,firstpub,tm_author,tm_title,us_national,authof3ormore,age
137,uc2.ark+=13960=fk3ws8hq14,1849.0,1916,"Allen, James Lane",A cathedral singer,True,True,67.0
19943,nyp.33433076044449,1875.0,1916,"Scott, Leroy",Partners of the night,False,False,41.0
19984,uc2.ark+=13960=t87h1h26q,1890.0,1916,"Sutherland, Joan",The edge of empire,False,False,26.0
4883,uc2.ark+=13960=t3804z70b,1865.0,1916,"Hill, Grace Livingston",A voice in the wilderness,True,True,51.0
19917,uc2.ark+=13960=t9p26rm6h,1880.0,1916,"Carlsen, C. L. (Carl Laurence)",The taming of Calinga,False,False,36.0


In [55]:
len(set(missing.docid).intersection(set(complete.docid)))

0

In [56]:
missing = missing.loc[ : , ['docid', 'tm_author', 'tm_title']]

In [57]:
missing['obscure'] = True

In [58]:
newcomplete = pd.concat([complete, missing])

In [59]:
newcomplete.shape

(5626, 11)

In [61]:
newcomplete.tail()

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller,obscure
1161,mdp.39015008633334,"Braine, John",Room at the top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
11199,uc1.$b104483,"Yates, Elizabeth",Guardian heart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1283,uc1.$b322859,"Brooke, Jocelyn",The image of a drawn sword,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
9145,uc1.32106014579582,"Shute, Nevil",A town like Alice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
8751,osu.32435017549254,"Ruck, Berta",Joyful journey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [62]:
tomerge = meta.loc[: , ['docid', 'firstpub', 'birthyear', 'us_national']]

In [63]:
newcomplete = newcomplete.merge(tomerge, how = 'inner', on = 'docid')
newcomplete.shape

(5626, 14)

In [64]:
newcomplete.head()

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller,obscure,firstpub,birthyear,us_national
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",False,NaN,False,NaN,1942,1913.0,True
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",False,NaN,False,NaN,1927,1867.0,False
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,False,NaN,False,NaN,1944,1887.0,True
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,False,NaN,False,NaN,1939,1873.0,True
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",False,NaN,False,NaN,1926,1874.0,True


In [65]:
newcomplete.is_bestseller = newcomplete.is_bestseller.fillna(value = False)
newcomplete.prize_author = newcomplete.prize_author.fillna(value = False)
newcomplete.obscure = newcomplete.obscure.fillna(value = False)

In [69]:
lowest = []
errors = 0
for idx, row in newcomplete.iterrows():
    values = []
    if not pd.isnull(row.tm_year):
        values.append(int(row.tm_year))
    if not pd.isnull(row.review_year):
        values.append(int(row.review_year))    
    if not pd.isnull(row.firstpub):
        values.append(int(row.firstpub))
    if not pd.isnull(row.bestseller_year):
        values.append(int(row.bestseller_year))
    
    minval = min(values)
    
    if minval < 1916 or minval > 1950:
        minval = 0
        errors += 1
    
    if not pd.isnull(row.review_year) and minval > 0 and row.review_year > minval + 5:
        print(row.review_year, minval)
        minval = 0
        errors += 1

    lowest.append(minval)
    
print(errors)

1927.0 1920
1927.0 1916
1948.0 1941
1935.0 1928
1933.0 1923
1941.0 1925
1947.0 1924
1949.0 1930
1950.0 1932
1950.0 1932
1949.0 1937
1940.0 1934
1942.0 1934
1950.0 1944
1942.0 1926
1943.0 1932
1938.0 1928
1941.0 1921
1950.0 1936
1944.0 1937
1948.0 1922
1932.0 1922
1929.0 1922
1931.0 1924
1946.0 1938
1927.0 1916
1936.0 1924
1945.0 1933
1942.0 1931
1937.0 1930
1934.0 1922
1929.0 1921
1927.0 1919
1932.0 1920
1949.0 1935
1927.0 1920
1931.0 1921
1938.0 1926
1928.0 1920
1936.0 1919
1949.0 1916
107


In [70]:
newcomplete['lowest_date'] = lowest

In [72]:
newcomplete = newcomplete.loc[newcomplete.lowest_date > 0, : ]
newcomplete.shape

(5519, 15)

In [81]:
newcomplete.tm_author = newcomplete.tm_author.str.lower()
newcomplete.tm_title = newcomplete.tm_title.str.lower()

In [85]:
standards = []

for auth1 in tqdm(newcomplete.tm_author):
    if pd.isnull(auth1) or len(auth1) < 1:
        auth1 = 'anonymous'
    auth1 = auth1.replace('[', '')
    auth1 = auth1.replace(']', '')
    if auth1.endswith('mrs'):
        auth1 = auth1[0: -3]
    if len(auth1) > 20:
        auth1 = auth1[0: 20]
        
    if auth1 in standards:
        standards.append(auth1)
    else:
        thetuples = []
        for auth2 in standards:
            match = fuzz.ratio(auth1, auth2)
            if match > 90:
                thetuples.append((match, auth2))
        
        if len(thetuples) < 1:
            standards.append(auth1)
        else:
            thetuples.sort(reverse = True)
            standards.append(thetuples[0][1])
            print(auth1, thetuples[0][1])
                

 26%|██▌       | 1441/5519 [00:02<00:09, 440.92it/s]

howard, elizabeth howard, elizabeth me
howard, elizabeth howard, elizabeth me


 30%|██▉       | 1636/5519 [00:02<00:09, 429.33it/s]

fielding, archibald fielding, archibald 
fielding, archibald fielding, archibald 
fielding, archibald fielding, archibald 


 34%|███▍      | 1864/5519 [00:03<00:12, 300.97it/s]

...

 74%|███████▍  | 4096/5519 [00:11<00:05, 249.87it/s]

fielding, archibald fielding, archibald 


 82%|████████▏ | 4531/5519 [00:13<00:08, 117.22it/s]

davis, robert h davis, robert
sheldon, charles m sheldon, charles mon


100%|██████████| 5519/5519 [00:21<00:00, 252.33it/s]


In [86]:
len(standards)

5519

In [87]:
newcomplete['standard_auth'] = standards

In [88]:
newcomplete.head(15)

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller,obscure,firstpub,birthyear,us_national,lowest_date,standard_auth
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",False,NaN,False,False,1942,1913.0,True,1942,"shaw, irwin"
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",False,NaN,False,False,1927,1867.0,False,1927,"bennett, arnold"
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,False,NaN,False,False,1944,1887.0,True,1944,"lippincott, joseph w"
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,False,NaN,False,False,1939,1873.0,True,1939,"becker, may lamberto"
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",False,NaN,False,False,1926,1874.0,True,1926,"miller, alice duer,"
5,mdp.39015002756842,"saroyan, william",daring young man on the flying,1934.0,1934.0,821.0,"the daring young man on the flying trapeze, an...",False,NaN,False,False,1934,1908.0,True,1934,"saroyan, william"
6,umn.31951000955121j,"chase, mary ellen",girl from,1916.0,1916.0,2149.0,"girl from the big horn country, il",False,NaN,False,False,1916,1887.0,False,1916,"chase, mary ellen"
7,mdp.39015063945169,"miln, louise (jordan), mrs","ann zu-zan, a chine",1932.0,1932.0,61.0,"ann zu-zan, a chinese love story.",False,NaN,False,False,1932,1864.0,True,1932,"miln, louise (jordan"
8,uc1.b3835561,"gogarty, oliver st. john",as i was goin,1937.0,1937.0,2770.0,as i was going down sackville street.,False,NaN,False,False,1937,1878.0,False,1937,"gogarty, oliver st."
9,mdp.39015028779778,"ferguson, j. de lancey (john de lancey)","pride and passion, robert burn",1939.0,1939.0,3524.0,"pride and passion, robert burns, 1759-1796.",False,NaN,False,False,1939,1888.0,False,1939,"ferguson, j. de lanc"


In [89]:
len(set(newcomplete.tm_author))

1927

In [91]:
len(set(newcomplete.standard_auth))

1918

In [92]:
meta = pd.read_csv("../../cohort/topicdata/comprehensivebooktopicdata.tsv", sep="\t", low_memory=False)

In [94]:
newstandards = []

for auth1 in tqdm(meta.hathi_author):
    if pd.isnull(auth1) or len(auth1) < 1:
        auth1 = 'anonymous'
    else:
        auth1 = auth1.lower()
    
    auth1 = auth1.replace('[', '')
    auth1 = auth1.replace(']', '')
    if auth1.endswith('mrs'):
        auth1 = auth1[0: -3]
    if len(auth1) > 20:
        auth1 = auth1[0: 20]
        
    if auth1 in standards:
        newstandards.append(auth1)
    else:
        thetuples = []
        for auth2 in standards:
            match = fuzz.ratio(auth1, auth2)
            if match > 90:
                thetuples.append((match, auth2))
        
        if len(thetuples) < 1:
            standards.append(auth1)
            newstandards.append(auth1)
        else:
            thetuples.sort(reverse = True)
            standards.append(thetuples[0][1])
            newstandards.append(thetuples[0][1])
            print(auth1, thetuples[0][1])

 11%|█         | 3095/29341 [00:12<02:54, 150.36it/s]

ellin, stanley elkin, stanley
ellin, stanley elkin, stanley


 12%|█▏        | 3391/29341 [00:13<01:38, 263.43it/s]

fielding, archibald fielding, archibald 
fielding, archibald fielding, archibald 
fielding, archibald fielding, archibald 
fielding, archibald fielding, archibald 


 13%|█▎        | 3848/29341 [00:14<01:27, 291.32it/s]

...

 99%|█████████▉| 29064/29341 [09:12<00:14, 19.52it/s]

carter, william carpenter, william


 99%|█████████▉| 29074/29341 [09:12<00:15, 17.17it/s]

anderson, david d anderson, david


100%|██████████| 29341/29341 [09:27<00:00, 51.73it/s]


In [95]:
len(newstandards)

29341

In [96]:
newcomplete.shape

(5519, 16)

In [98]:
newcomplete.to_csv('allmatchedbooks.tsv', sep = '\t', index = False)

In [99]:
len(set(newstandards))

12370

In [100]:
len(set(meta.hathi_author))

12532

In [101]:
meta.shape

(29341, 208)

In [102]:
newcomplete.head()

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller,obscure,firstpub,birthyear,us_national,lowest_date,standard_auth
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",False,NaN,False,False,1942,1913.0,True,1942,"shaw, irwin"
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",False,NaN,False,False,1927,1867.0,False,1927,"bennett, arnold"
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,False,NaN,False,False,1944,1887.0,True,1944,"lippincott, joseph w"
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,False,NaN,False,False,1939,1873.0,True,1939,"becker, may lamberto"
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",False,NaN,False,False,1926,1874.0,True,1926,"miller, alice duer,"


In [103]:
meta['standard_auth'] = newstandards

In [104]:
meta.tail()

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,...,t199,docid,birthyear,firstpub,hathi_author,hathi_title,us_national,authof3ormore,age,standard_auth
29336,0.004679,0.000006,0.000409,0.046729,0.000439,0.000006,0.000003,0.000010,0.001933,0.052563,...,0.000002,mdp.39015043059172,1932.0,1999,"Smithies, Michael",Gulfs of Thailand : a collection of short stories,False,False,67.0,"smithies, michael"
29337,0.000111,0.001208,0.000319,0.010090,0.000717,0.000409,0.000005,0.000016,0.000014,0.000113,...,0.000703,mdp.39015047842185,NaN,1999,"Claxton, William",Laugh : portraits of the greatest comedians an...,False,False,NaN,"claxton, william"
29338,0.000050,0.000027,0.001393,0.000329,0.009165,0.000376,0.000160,0.000019,0.000131,0.006394,...,0.000081,mdp.39015051894841,NaN,1999,"Keyes, Marian",Last chance saloon,False,False,NaN,"keyes, marian"
29339,0.000611,0.002533,0.007632,0.003172,0.007383,0.000112,0.000002,0.000187,0.000292,0.000292,...,0.000002,mdp.39015047609089,NaN,1999,"Mayo, Wendell",B. Horror : and other stories,False,False,NaN,"mayo, wendell"
29340,0.000164,0.000006,0.004271,0.000004,0.027734,0.000006,0.000003,0.003068,0.000576,0.000198,...,0.000002,mdp.39015046006196,NaN,1999,"Vega, Suzanne",The passionate eye : the collected writing of ...,False,False,NaN,"vega, suzanne"


In [105]:
meta = meta.loc[(meta.firstpub > 1885) & (meta.firstpub < 1977), :]
meta.shape

(22850, 209)

In [106]:
meta.to_csv('topicdatastandardauths.tsv', sep = '\t', index = False)

In [107]:
yeardict = dict()

for year, df in newcomplete.groupby('lowest_date'):
    yeardict[year] = df

In [108]:
len(yeardict)

35

In [109]:
del yeardict